In [276]:
import scipy, math
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [277]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
isolet = fetch_ucirepo(id=54) 
  
# data (as pandas dataframes) 
X = isolet.data.features
y = isolet.data.targets 

In [591]:
# Transforming the dataframes into numpy arrays (matrices with real entries)
X_array = np.array(X).T
Y_array = OneHotEncoder().fit_transform(y).toarray().T

In [592]:
# Standardizing the data to have feature mean 0 with standard deviation of 1
X_std = (X_array - X_array.mean(1,keepdims=True))/X_array.std(1,keepdims = True)
Y_std = (Y_array - Y_array.mean(1,keepdims=True))/Y_array.std(1,keepdims = True)

In [593]:
X_array.shape

(617, 7797)

In [594]:
Y_array.shape

(26, 7797)

In [595]:
# n = number of features, m = number of observations, k = number of classes
n,m = X_array.shape
k = Y_array.shape[0]

We'll look for an optimal transformation matrix $W$ such that $W^TX \approx Y$ through a least squares approach:
$$ \min_{W \in \mathbb{R}^{n \times k}} \  \{ f(W) \coloneqq \lVert W^TX - Y \rVert_F^2 + \lambda \lVert W \rVert_F^2 \}, $$
where 
- $X \in \mathbb{R}^{n \times m}$ is a data matrix.
- $Y \in \mathbb{R}^{k \times m}$ is a label matrix (one-hot-encoded matrix).
- $W \in \mathbb{R}^{n \times k}$ is a transformation matrix.  
- $\lambda > 0$ is a regularization hyperparameter.
- $\lVert \cdot \rVert_F$ is the Frobenius norm.

In [631]:
# We'll split the datasets X and Y into "training" and "testing" with a 70/30 split.
m_train = int(np.ceil(m*0.7))
m_test = int(m - m_train)

# Randomly select columns (observations) to be a part of the training set.  
train_index = np.random.choice(X_array.shape[1], m_train, replace = False)
test_index = list(set(range(m)) - set(train_index))

In [598]:
X_train = X_std[:,train_index]; Y_train = Y_std[:,train_index]
X_test = X_std[:,test_index]; Y_test = Y_array[:,test_index]

The problem $$\min_{W \in \mathbb{R}^{n \times k}} \  \{ f(W) \coloneqq \lVert W^TX - Y \rVert_F^2 + \lambda \lVert W \rVert_F^2 \}$$ is unconstrained.  Therefore, we can solve for $W$ directly with basic calculus.  

First, we'll re-write $f(W)$ in its equivalent form by noting that for any matrix $C$ we have $\lVert C \rVert_F^2 = \text{trace}(C^TC)$, where $\text{trace}$ is the trace operator:  

$$ f(W) = \lVert Y \rVert_F^2 + \text{trace}(W^TXX^TW) - 2\text{trace}(W^TXY^T) + \lambda \text{trace}(W^TW)$$

Next, we can set the partial derivative of $f(W)$ with respect to $W$ equal to $0$, and solve for $W$:

$$ \frac{\partial f}{\partial W} = 2XX^TW - 2XY^T + 2\lambda W = 0 $$
$$ \iff (XX^T + \lambda I_n)W = XY^T $$ 
$$ \Rightarrow W = (XX^T + \lambda I_n)^{-1}(XY^T) $$

In [623]:
# Solving for an optimal W given the data matrix X, label matrix Y, and lambda_val.  
lambda_val = 100
In = np.eye(n)
W = scipy.linalg.inv(X_train@X_train.T + lambda_val*In)@(X_train@Y_train.T)

In [602]:
# Calculating estimate matrix Y_est and creating its label equivalent Y_est_label
Y_est = W.T@X_test
Y_est_label = np.zeros((k,n_test))

In [625]:
# For each obsevation, look at the index (class) which has the highest value and consider that its classifcation.  
for i in range(n_test):
    temp_list = list(Y_est[:,i])
    max_index = temp_list.index(max(temp_list))
    Y_est_label[max_index,i] = 1

In [604]:
# Transform one-hot-encoded matrices back to vectors with class labels (1,2,...,k).  
test_labels = np.argmax(Y_test, axis = 0)
est_labels = np.argmax(Y_est_label, axis = 0)

In [627]:
# Calculate accuracy
agreement = (test_labels == est_labels)*1
accuracy = (sum(agreement)/n_test)*100

In [629]:
print(accuracy)

92.90294997862334
